::: {.callout-note}
## Objectives

This guide demonstrates AE specific analysis showing individual adverse event terms. For design philosophy and architecture details, see [ae_summary.qmd](ae_summary.qmd).
:::

## Setup


In [ ]:
import sys
from pathlib import Path
import polars as pl

sys.path.insert(0, 'src')

from rtflite import LibreOfficeConverter
try:
    converter = LibreOfficeConverter()
except:
    converter = None
    print("WARNING: LibreOffice not found. PDF conversion will be skipped.")

In [ ]:
from csrlite import load_plan, study_plan_to_ae_specific
from csrlite.ae.ae_specific import ae_specific_ard, ae_specific_df, ae_specific_rtf, ae_specific

## What's Different?

**AE Summary** shows high-level event categories (e.g., "Any AE", "Serious AE").

**AE Specific** shows detailed listings of individual adverse event terms (e.g., "APPLICATION SITE ERYTHEMA", "DIARRHOEA").

## StudyPlan-Driven Workflow


In [ ]:
study_plan = load_plan("studies/xyz123/yaml/plan_xyz123.yaml")
study_plan.get_plan_df().filter(pl.col("analysis") == "ae_specific")

In [ ]:
output_files = study_plan_to_ae_specific(study_plan)

In [ ]:
#| echo: false
for file in output_files:
    if converter:
        converter.convert(file, output_dir="docs/pdf/", format="pdf", overwrite=True)

### week12_any

<embed src="pdf/ae_specific_apat_week12_any.pdf" style="width:100%; height:600px" type="application/pdf">

### week12_rel

<embed src="pdf/ae_specific_apat_week12_rel.pdf" style="width:100%; height:600px" type="application/pdf">

### week12_ser

<embed src="pdf/ae_specific_apat_week12_ser.pdf" style="width:100%; height:600px" type="application/pdf">

### week24_any

<embed src="pdf/ae_specific_apat_week24_any.pdf" style="width:100%; height:600px" type="application/pdf">

### week24_rel

<embed src="pdf/ae_specific_apat_week24_rel.pdf" style="width:100%; height:600px" type="application/pdf">

### week24_ser

<embed src="pdf/ae_specific_apat_week24_ser.pdf" style="width:100%; height:600px" type="application/pdf">

## Complete Pipeline


In [ ]:
adsl = pl.read_parquet("data/adsl.parquet")
adae = pl.read_parquet("data/adae.parquet")

ae_specific(
    population=adsl,
    observation=adae,
    population_filter="SAFFL = 'Y'",
    observation_filter="TRTEMFL = 'Y'",
    parameter_filter=None,
    id=("USUBJID", "Subject ID"),
    group=("TRT01A", "Treatment Group"),
    ae_term=("AEDECOD", "Adverse Event"),
    title=[
        "Analysis of Specific Adverse Events",
        "(Treatment-Emergent, Safety Population)"
    ],
    footnote=[
        "Every participant is counted a single time for each applicable row and column."
    ],
    source=["Source: ADSL and ADAE datasets"],
    output_file="studies/xyz123/rtf/ae_specific.rtf",
    total=True,
    missing_group="error"
)

In [ ]:
#| echo: false
if converter:
    converter.convert(f"{study_plan.output_dir}/ae_specific.rtf", output_dir="docs/pdf/", format="pdf", overwrite=True)

<embed src="pdf/ae_specific.pdf" style="width:100%; height:600px" type="application/pdf">

## Step-by-Step Pipeline

### Step 1: Generate ARD

**Key Parameters:**
- `ae_term`: Tuple (variable_name, label) for AE term column (default: ("AEDECOD", "Adverse Event"))
- `parameter_filter`: Optional SQL WHERE clause for filtering AEs (e.g., "AESER = 'Y'" for serious AEs, "AEREL in ['RELATED', 'PROBABLY RELATED']" for related AEs)


In [ ]:
_ard = ae_specific_ard(
    population=adsl,
    observation=adae,
    population_filter="SAFFL = 'Y'",
    observation_filter="TRTEMFL = 'Y'",
    parameter_filter=None,
    id=("USUBJID", "Subject ID"),
    group=("TRT01A", "Treatment Group"),
    ae_term=("AEDECOD", "Adverse Event"),
    total=True,
    missing_group="error"
)

_ard

**Output Structure:** Long format with `__index__`, `__group__`, `__value__` columns.

### Step 2: Transform to Display Format


In [ ]:
_df = ae_specific_df(_ard)
_df

**Output Structure:** Wide format with "Term" column showing AE terms and treatment groups as columns.

### Step 3: Generate RTF Output


In [ ]:
ae_specific_rtf(
    _df,
    title=[
        "Analysis of Specific Adverse Events",
        "(Treatment-Emergent, Safety Population)"
    ],
    footnote=[
        "Every participant is counted a single time for each applicable row and column."
    ],
    source=["Source: ADSL and ADAE datasets"],
    col_rel_width=[4, 2, 2, 2, 2]
).write_rtf("studies/xyz123/rtf/ae_specific_step.rtf")

In [ ]:
#| echo: false
if converter:
    converter.convert(f"{study_plan.output_dir}/ae_specific_step.rtf", output_dir="docs/pdf/", format="pdf", overwrite=True)

<embed src="pdf/ae_specific_step.pdf" style="width:100%; height:600px" type="application/pdf">